# <center> Variational Post-selection
## <center> TFIM

## Setup

In [ ]:
import numpy as np
import time
import tensorflow as tf
import tensorcircuit as tc
from tensorcircuit.applications.graphdata import Grid2D
from tensorcircuit.applications.vags import tfim_measurements_tc
import cotengra
import csv
import scipy

optc = cotengra.ReusableHyperOptimizer(
    methods=["greedy"],
    parallel="ray",
    minimize="combo",
    max_time=30,
    max_repeats=1024,
    progbar=True,
)
tc.set_contractor("custom", optimizer=optc, preprocessing=True)

K = tc.set_backend("tensorflow")
tc.set_dtype("complex128")

In [ ]:
ns = 12
na = 1
n = ns + na
g = Grid2D(4, 3)

In [ ]:
g1 = tc.templates.graphs.Grid2DCoord(4, 3).lattice_graph(pbc=True)
h = tc.quantum.heisenberg_hamiltonian(g1, hzz=1.0, hxx=0.0, hyy=0.0, hx=-1.0, sparse = True, numpy=True)
e_anal = min(scipy.sparse.linalg.eigsh(h, which='SA')[0])
e_anal

## Circuit

In [ ]:
def U(n, depth, params):
    params = K.cast(params, "complex128")
    c = tc.Circuit(n)
    idx = 0

    for i in range(n):
        c.H(i)

    for _ in range(depth):
        for i in range(n-1):
            c.rzz(i, (i + 1) % n, theta=params[idx+i])
        idx+=n-1

        for i in range(n):
            c.rx(i, theta=params[idx+i])
        idx+=n

    return c, idx

In [ ]:
# circuit visulization, optional
p = 1
cirq, idx = U(ns, p, np.zeros([1000]))
print("The number of parameters is", idx)
cirq.draw()

## Conventional VQE

In [ ]:
def train(ns, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U(ns, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c, _ = U(ns, p, params)
            e = tfim_measurements_tc(c, g, hzz=1.0, hx=-1.0)
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        grads = tape.gradient(e, params)
        opt.apply_gradients(zip([grads], [params]))
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}".format(i, e_list[-1], re_list[-1]))
    
    params_list.append(params.numpy())
    print(params.numpy())
    
    return e_list, re_list, params_list

In [ ]:
p = 3
maxiter = 1000
lr = 0.1
stddev = np.pi
t = time.gmtime()

# train
with tf.device("/cpu:0"):
    with open('data/tfim_p{}_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp1:   # check the filename
        with open('data/tfim_p{}_params_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp2:   # check the filename
            writer1 = csv.writer(fp1)
            writer2 = csv.writer(fp2)
            for j in range(50):
                print(j+1)
                e_list, re_list, params_list = train(ns, p, e_anal, maxiter=maxiter, lr=lr, stddev=stddev)
                writer1.writerow([j])
                writer1.writerow(e_list)
                writer1.writerow(re_list)
                writer2.writerow([j])
                for k in range(maxiter+1):
                    writer2.writerow(params_list[k])

## Post-selected VQE

In [ ]:
def postselect(n, ns, p, params):
    c, _ = U(n, p, params)

    w = c.wavefunction()[ : 2**ns]
    norm = tf.linalg.norm(w)
    w = w / norm
    
    return tc.Circuit(ns, inputs=w), np.abs(norm)**2

In [ ]:
def train(n, ns, na, p, e_anal, maxiter=10000, lr=0.01, stddev=1.0):
    _, idx = U(n, p, np.zeros([1000]))
    params = tf.Variable(
        initial_value=tf.random.normal(
            shape=[idx], stddev=stddev, dtype=getattr(tf, tc.rdtypestr)
        )
    )

    exp_lr = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr, decay_steps=150, decay_rate=0.7
    )
    opt = tf.keras.optimizers.Adam(exp_lr)

    e_list = []
    re_list = []
    params_list = []
    ps_list = []
    for i in range(maxiter):
        with tf.GradientTape() as tape:
            c_s, norm = postselect(n, ns, p, params)
            e = tfim_measurements_tc(c_s, g, hzz=1.0, hx=-1.0)
            ps_list.append(norm)
        e_list.append(K.real(e).numpy())
        re_list.append(np.abs((e_list[-1] - e_anal) / e_anal))
        params_list.append(params.numpy())
        grads = tape.gradient(e, params)
        opt.apply_gradients(zip([grads], [params]))
        if (i + 1) % 50 == 0 or i==0:
            print("epoch{:>4}, e: {:.10f}, re: {:.10f}".format(i, e_list[-1], re_list[-1]))
    
    params_list.append(params.numpy())
    print(params.numpy())

    return e_list, re_list, params_list, ps_list

In [ ]:
p = 3
maxiter = 1000
lr = 0.1
stddev = np.pi
t = time.gmtime()

# train
with tf.device("/cpu:0"):
    with open('data/tfim_ps_p{}_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp1:   # check the filename
        with open('data/tfim_ps_p{}_params_{}.csv'.format(p, time.strftime("%Y_%m%d_%H_%M_%S",t)), 'w', newline='') as fp2:   # check the filename
            writer1 = csv.writer(fp1)
            writer2 = csv.writer(fp2)
            for j in range(50):
                print(j+1)
                e_list, re_list, params_list, ps_list = train(n, ns, na, p, e_anal, maxiter=maxiter, lr=lr, stddev=stddev)
                writer1.writerow([j])
                writer1.writerow(e_list)
                writer1.writerow(re_list)
                writer1.writerow(ps_list)

                writer2.writerow([j])
                writer2.writerows(params_list)